In [4]:
from skimage.transform import resize
from skimage.color import rgb2gray
from imageio import imread
import numpy as np
import matplotlib.pyplot as plt

FILE_PATH = './raw/jpg/Kolo.jpg'

In [5]:
from skimage.transform import radon, iradon
def load_image(file_path, size=128):
    image = imread(file_path)
    if image.ndim == 3:  # Convert to grayscale if it's a color image
        image = rgb2gray(image)
    image = resize(image, (size, size), mode='reflect', anti_aliasing=True)
    return image

# Niestandardowa implementacja transformaty Radona
def custom_radon(image, theta):
    size = image.shape[0]
    num_angles = len(theta)
    sinogram = np.zeros((num_angles, size))
    center = size // 2
    max_s = size * np.sqrt(2) / 2
    
    x = np.arange(size) - center
    y = np.arange(size) - center
    X, Y = np.meshgrid(x, y)
    
    for i, angle in enumerate(np.deg2rad(theta)):
        cos_a = np.cos(angle)
        sin_a = np.sin(angle)
        
        # Oblicz współrzędne s dla wszystkich pikseli
        S = X * cos_a + Y * sin_a
        
        # Interpolacja liniowa do skorelowania z pozycjami detektora
        s_indices = ((S + max_s) * (size - 1) / (2 * max_s))
        s_indices = np.clip(s_indices, 0, size - 1)
        
        # Sumowanie wartości pikseli
        for s in range(size):
            mask = (np.floor(s_indices) <= s) & (s < np.ceil(s_indices))
            sinogram[i, s] += image[mask].sum()
            
    return sinogram / size

# Filtr ramp dla odwrotnej transformacji
def ramp_filter(size):
    n = np.arange(-size//2, size//2)
    h = np.zeros_like(n, dtype=np.float32)
    h[n == 0] = 0.25
    h[n % 2 == 1] = -1 / (np.pi * n[n % 2 == 1])**2
    return np.fft.fftshift(h)

# Niestandardowa odwrotna transformata Radona
def custom_iradon(sinogram, theta, output_size):
    size = sinogram.shape[1]
    num_angles = len(theta)
    
    # Filtrowanie sinogramu
    filt = ramp_filter(size)
    filtered = np.zeros_like(sinogram)
    for i in range(num_angles):
        filtered[i] = np.convolve(sinogram[i], filt, mode='same')
    
    # Backprojection
    reconstruction = np.zeros((output_size, output_size))
    center = output_size // 2
    x = np.arange(output_size) - center
    y = np.arange(output_size) - center
    X, Y = np.meshgrid(x, y)
    
    for i, angle in enumerate(theta):
        angle_rad = np.deg2rad(angle)
        cos_a = np.cos(angle_rad)
        sin_a = np.sin(angle_rad)
        
        S = X * cos_a + Y * sin_a
        s_indices = (S * (size / (output_size * np.sqrt(2))) + size/2).clip(0, size-1)
        
        projection = np.interp(s_indices.flatten(), 
                             np.arange(size), 
                             filtered[i]).reshape(output_size, output_size)
        
        reconstruction += projection
    
    return reconstruction * (np.pi / (2 * num_angles))

In [ ]:
input_image = load_image(FILE_PATH)

theta = np.arange(180)
sinogram = custom_radon(input_image, theta=theta)

reconstructed = custom_iradon(sinogram, theta=theta, output_size=input_image.shape[0])

reconstructed = np.clip(reconstructed, 0, 1)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(input_image, cmap='gray', vmin=0, vmax=1)
axes[0].set_title('Obraz wejściowy')
axes[0].axis('off')

axes[1].imshow(sinogram, cmap='gray', aspect='auto', 
              extent=(0, 180, 0, sinogram.shape[0]))
axes[1].set_title('Sinogram')
axes[1].set_xlabel('Kąt projekcji (stopnie)')
axes[1].set_ylabel('Pozycja detektora (piksele)')

axes[2].imshow(reconstructed, cmap='gray', vmin=0, vmax=1)
axes[2].set_title('Obraz odtworzony')
axes[2].axis('off')

plt.tight_layout()
plt.show()

C:\Users\maati\AppData\Local\Temp\ipykernel_15128\3161852923.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(file_path)


IndexError: index 128 is out of bounds for axis 0 with size 128